In [23]:
from pyspark.sql import SparkSession

In [24]:
spark = SparkSession.builder.appName("s").getOrCreate()

In [25]:
data = spark.read.csv("data/titanic.csv", header=True, inferSchema=True)
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [26]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [27]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [28]:
data = data.select(['Survived','Pclass','Sex', 'Age',
 'SibSp','Parch','Fare','Embarked'])

In [29]:
data = data.na.drop()

### Working With categorical Columns

In [30]:
from pyspark.ml.feature import (StringIndexer,VectorAssembler,
                                VectorIndexer, OneHotEncoder)

In [31]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol="SexVec")

In [32]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVector')

In [33]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVector'], outputCol='features')

In [34]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [35]:
lr = LogisticRegression(featuresCol='features',labelCol='Survived')

In [36]:
pipeline = Pipeline(stages=[gender_indexer, gender_encoder,
                            embark_indexer, embark_encoder,
                           assembler, lr])

In [37]:
train_data, test_data = data.randomSplit([0.7, 0.3])

In [38]:
fit_model = pipeline.fit(train_data)

Exception ignored in: <function JavaWrapper.__del__ at 0x000001B17FEFD280>
Traceback (most recent call last):
  File "C:\Users\Ahmed-Basem\Anaconda3\lib\site-packages\pyspark\ml\wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LogisticRegression' object has no attribute '_java_obj'


In [39]:
results = fit_model.transform(test_data)

In [40]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [46]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [47]:
results.select(['prediction','Survived']).show()

+----------+--------+
|prediction|Survived|
+----------+--------+
|       1.0|       0|
|       1.0|       0|
|       1.0|       0|
|       1.0|       0|
|       1.0|       0|
|       0.0|       0|
|       1.0|       0|
|       1.0|       0|
|       1.0|       0|
|       0.0|       0|
|       0.0|       0|
|       1.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
+----------+--------+
only showing top 20 rows



In [48]:
AUC = my_eval.evaluate(results)

In [49]:
AUC

0.7793166035353536